In [61]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [62]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [63]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [64]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
engine

Engine(sqlite:///Resources/hawaii.sqlite)

In [65]:
# Declare a Base using `automap_base()`
Base = automap_base()
# reflect an existing database into a new model
Base.prepare(engine, reflect=True)
# reflect the tables
Base.classes.keys()

['measurement', 'station']

In [66]:
# View all of the classes that automap found
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [67]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [68]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [69]:
# Find the most recent date in the data set.
query = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
query.date

'2017-08-23'

In [70]:
# Verify Measurement structure
columns = inspector.get_columns('Measurement')
columns

[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'station',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'date',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'prcp',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'tobs',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

In [101]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
date = dt.datetime.strptime(query.date, "%Y-%m-%d") - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement).filter(Measurement.date > date)


# Save the query results as a Pandas DataFrame and set the index to the date column
dfres = pd.read_sql(results.statement, results.session.bind)

# Sort the dataframe by date
## While I think sqlalchemy returned a result sorted by date, I will sort it again for kicks
## Oh and that date isn't a date, its an object
fix_dfres = pd.to_datetime(dfres["date"])
dfres['date'] = fix_dfres
sorted_dfres = dfres.sort_values(by='date')

# Use Pandas Plotting with Matplotlib to plot the data


#dfres.dtypes
sorted_dfres

,id,station,date,prcp,tobs
0,2365,USC00519397,2016-08-24,0.08,79.0
1006,8826,USC00517948,2016-08-24,NaN,78.0
1524,14609,USC00519281,2016-08-24,2.15,77.0
702,7332,USC00514830,2016-08-24,2.28,80.0
360,5092,USC00513117,2016-08-24,2.15,76.0
...,...,...,...,...,...
1522,12186,USC00519523,2017-08-22,0.00,82.0
1523,12187,USC00519523,2017-08-23,0.08,82.0
359,2724,USC00519397,2017-08-23,0.00,81.0
1005,7635,USC00514830,2017-08-23,0.00,82.0


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()